In [14]:
from cgeniepy.ecology import EcoModel
import pandas as pd
from cgeniepy.array import GriddedData
from pathlib import Path
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# model_path = Path("../model")

# ## list all folders containing "EXP1'
# exp1 = [str(x) for x in model_path.iterdir() if x.is_dir() and "EXP1" in x.name]
# exp2 = [str(x) for x in model_path.iterdir() if x.is_dir() and "EXP2" in x.name]
# exp3 = [str(x) for x in model_path.iterdir() if x.is_dir() and "EXP3" in x.name]

m1 = EcoModel('../model/muffin.u067bc.PO4.8P8Z.EXP1', gemflag=['biogem', 'ecogem'])
m2 = EcoModel('../model/muffin.u067bc.PO4.8P8Z.EXP2', gemflag=['biogem', 'ecogem'])
m3 = EcoModel('../model/muffin.u067bc.PO4.8P8Z.EXP3', gemflag=['biogem', 'ecogem'])

def ocn_var_mean_change(model, variable, anomaly=True, timeslice=[1,2]):
    if anomaly:
        var0 = model.get_var(variable).isel(time=0).data
        var1 = model.get_var(variable).isel(time=timeslice).mean(dim=['time']).data
        diff = var1 - var0
        return GriddedData(diff, attrs=model.get_var(variable).attrs)
    else:
        var1 = model.get_var(variable).isel(time=timeslice).mean(dim=['time']).data
        return GriddedData(var1, attrs=model.get_var(variable).attrs)
    
##　use Helveticas
plt.rcParams['font.sans-serif'] = "Helvetica"

18

In [23]:
total_biomass = m3.get_var("eco2D_Plankton_C_Total")
import numpy as np

data = []
for i in range(1, 23):
    biomass = m3.get_pft(i)
    pft_biomass_fraction = biomass / total_biomass
    mean_pft_biomass_fraction = pft_biomass_fraction.mean(dim=['lon','lat']).data.values
    ## copy the first value and insert it to the front
    mean_pft_biomass_fraction = np.insert(mean_pft_biomass_fraction, 0, mean_pft_biomass_fraction[0], axis=0)
    data.append(mean_pft_biomass_fraction)

In [89]:
## stacked aera plot
import numpy as np
data = np.array(data)

## stacked area plot
years = total_biomass.data.time
## add -100 to the first 
years = np.insert(years, 0, years[0]-10)

## concate two columns and use as label
labels = m3.eco_pars()['PFT'] + ', '+ m3.eco_pars()['diameter'].astype(str) + 'μm'

In [90]:
## create data frame
df= pd.DataFrame(data.T, columns=labels)
df['Year'] = years

## melt to long format
df = df.melt(id_vars='Year', var_name='PFT', value_name='Fraction')

In [91]:
## plot using lets-plot
from lets_plot import *
LetsPlot.setup_html()


p = ggplot(df, aes(x='Year', y='Fraction', fill='PFT')) + geom_area(alpha=0.8) + scale_fill_brewer(type='qual', palette='Set3') + theme(axis_text_x=element_text(angle=45)) + labs(x='Time after the impact (yr)', y='Fraction') + facet_wrap('PFT', order=0)
p = p + theme(legend_position='none', text=element_text(size=24))

ggsave(plot=p, filename="biomass_fraction.pdf", w=5, h=5, dpi=300, unit='in')

'/Users/yingrui/Science/kpg_ecosystem/code/lets-plot-images/biomass_fraction.pdf'

In [ ]:
mld = ocn_var_mean_change(m3, 'phys_MLD', True).to_GriddedDataVis()
poc = ocn_var_mean_change(m3, 'bio_fpart_POC', True).isel(zt=0).to_GriddedDataVis()
ph = ocn_var_mean_change(m3, 'misc_pH', True).to_GriddedDataVis()
omega_cal= ocn_var_mean_change(m3, 'carb_ohm_cal', False).isel(zt=0)
omega_cal.attrs['units'] = 'unitless'
omega_cal = omega_cal.to_GriddedDataVis()
omega_ara = ocn_var_mean_change(m3, 'carb_ohm_arg', False).isel(zt=0)
omega_ara.attrs['units'] = 'unitless'
omega_ara = omega_ara.to_GriddedDataVis()

sst = ocn_var_mean_change(m3, 'ocn_sur_temp', True).to_GriddedDataVis()
po4 = ocn_var_mean_change(m3, 'ocn_sur_PO4', True).to_GriddedDataVis()


In [ ]:
fig, axs = plt.subplots(3,3,figsize=(10, 10), subplot_kw={'projection': ccrs.Robinson()})

import matplotlib.colors as colors

sst.aes_dict['pcolormesh_kwargs']['cmap'] = 'RdBu_r'
sst.aes_dict['pcolormesh_kwargs']['norm'] = colors.CenteredNorm(halfrange=15)
sst.aes_dict['colorbar_label_kwargs'] = {'label': '°C'}
sst.plot(ax=axs[0,0], outline=True, colorbar=True)


po4.aes_dict['pcolormesh_kwargs']['cmap'] = 'RdBu_r'
po4.aes_dict['pcolormesh_kwargs']['norm'] = colors.CenteredNorm(halfrange=3E-6)
po4.aes_dict['colorbar_label_kwargs'] = {'label': 'mol kg$^{-1}$'}
po4.plot(ax=axs[0,1], outline=True, colorbar=True)

mld.aes_dict['pcolormesh_kwargs']['cmap'] = 'RdBu_r'
mld.aes_dict['pcolormesh_kwargs']['norm'] = colors.CenteredNorm(halfrange=2000)
mld.aes_dict['colorbar_label_kwargs'] = {'label': 'm'}
mld.plot(ax=axs[0,2], outline=True, colorbar=True)

poc.aes_dict['pcolormesh_kwargs']['cmap'] = 'RdBu_r'
poc.aes_dict['pcolormesh_kwargs']['norm'] = colors.CenteredNorm(halfrange=5)
poc.aes_dict['colorbar_label_kwargs'] = {'label': r'mol C m$^{-2}$  yr$^{-1}$'}
poc.plot(ax=axs[1,0], outline=True, colorbar=True)

ph.aes_dict['pcolormesh_kwargs']['cmap'] = 'RdBu_r'
ph.aes_dict['pcolormesh_kwargs']['norm'] = colors.CenteredNorm(halfrange=0.3)
ph.aes_dict['colorbar_label_kwargs'] = {'label': 'unitless'}
ph.plot(ax=axs[1,1], outline=True, colorbar=True)


omega_cal.aes_dict['pcolormesh_kwargs']['cmap'] = 'viridis'
omega_cal.aes_dict['pcolormesh_kwargs']['vmax'] = 3
omega_cal.aes_dict['colorbar_label_kwargs'] = {'label': 'unitless'}
omega_cal.aes_dict['contour_kwargs'] = {'levels': [1], 'colors': 'red'}
omega_cal.aes_dict['contour_label_kwargs'] = {'colors': 'red'}
omega_cal.plot(ax=axs[1,2], outline=True, colorbar=True, contour=True)


## add a contour ofr omega
omega_ara.aes_dict['pcolormesh_kwargs']['cmap'] = 'viridis'
omega_ara.aes_dict['pcolormesh_kwargs']['vmax'] = 3
omega_ara.aes_dict['colorbar_label_kwargs'] = {'label': 'unitless'}
omega_ara.aes_dict['contour_kwargs'] = {'levels': [1], 'colors': 'red'}
omega_ara.aes_dict['contour_label_kwargs'] = {'colors': 'red'}

omega_ara.plot(ax=axs[2,0], outline=True, colorbar=True, contour=True)

## add alphabetic labels for each subplot
for i, ax in enumerate(axs.flat):
    ax.text(0, 1.0, chr(97+i), transform=ax.transAxes, 
            fontsize=12, fontweight='bold', va='top')
    
## add title for each subplot
axs[0,0].set_title('Sea Surface Temperature')
axs[0,1].set_title('Surface Phosphate')
axs[0,2].set_title('Mixed Layer Depth')
axs[1,0].set_title('POC fluxes')
axs[1,1].set_title('Surface pH')
axs[1,2].set_title('Omega Calcite')
axs[2,0].set_title('Omega Aragonite')

## remove last subplot
fig.delaxes(axs[2,1])
fig.delaxes(axs[2,2])
fig.savefig('../output/env_change_map.png', dpi=300, bbox_inches='tight')

In [ ]:
# ## export for EXP4/5
# import numpy as np

# ## convert celsius to kelvin
# sst = ocn_var_change(m3, 'ocn_sur_temp', 0)
# sst_kelvin = sst.array.values + 273.15
# ## round to 2 decimal places
# ## replace na with 1E-5
# np.nan_to_num(sst_kelvin, copy=False, nan=1E-5)
# np.savetxt('../data/muffin.u067bc.PO4.SST.txt', sst_kelvin, fmt='%.2f', delimiter='\t')

# #PARfrac * dum_egbg_fxsw(:,:)
# fxsw = ocn_var_change(m3, 'phys_fxsw', 0)
# par_frac = 0.43
# par = fxsw * par_frac

# np.savetxt('../data/muffin.u067bc.PO4.PAR.txt', par.array.values, fmt='%.2f', delimiter='\t')